# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
# ! tells jupyter to execute as shell command
!pip install torch
!pip install torchvision
!pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Processing...


D:\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Then, we define the model, object function and optimizer that we use to classify.

In [4]:
import torch.optim as optim
import torch.nn.functional as F

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5,stride=1, padding=2)
        self.s1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5,stride=1)
        self.s2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5,stride=1)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.s1(x)
        x = F.relu(self.conv2(x))
        x = self.s2(x)
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Next, we can start to train and evaluate!

In [6]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        model.train()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
test_loss = 0.0
test_correct = 0
train_loss = 0.0
train_correct = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        test_loss += criterion(outputs, labels).item()
        # get the index of the max log-probability
        test_correct += torch.eq(labels, pred).float().sum()
        
with torch.no_grad():
    for images, labels in tqdm(train_loader):
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        train_loss += criterion(outputs, labels).item()
        # get the index of the max log-probability
        train_correct += torch.eq(labels, pred).float().sum()

print('\nTest Accuracy: {:.2f}%\n'.format(100.0 * test_correct / len(test_loader.dataset)))
print('\nTrain Accuracy: {:.2f}%\n'.format(100.0 * train_correct / len(train_loader.dataset)))
# print('\nTest Loss: {:.5f}\n'.format(test_loss / len(test_loader.dataset)))

100%|██████████| 468/468 [00:15<00:00, 30.51it/s]



Test Accuracy: 98.96%


Train Accuracy: 99.72%



#### Q5:
Please print the training and testing accuracy.